In [2]:
# Compatibility with Python 3
from __future__ import (absolute_import, division, print_function)

try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Basic Tools
import numpy as np
from astropy.table import Table
import pickle
import time
import matplotlib.pyplot as plt
import scipy.optimize as op
import sys
import time

# The Payne, see https://github.com/tingyuansen/The_Payne for more details
from The_Payne import training
from The_Payne import utils

In [15]:
# That's how we would do it:
# training_labels, training_spectra, validation_labels, validation_spectra = utils.load_training_data()
"""
Changes that need to be made to training.py in The_Payne if no CUDA is available

if torch.cuda.is_available():
    dtype = torch.cuda.FloatTensor
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    dtype = torch.FloatTensor
    torch.set_default_tensor_type('torch.FloatTensor')
    
if torch.cuda.is_available():
    model.cuda()
        
if torch.cuda.is_available():
    perm = perm.cuda()
"""
# That's how we do it:

"\nChanges that need to be made to training.py in The_Payne if no CUDA is available\n\nif torch.cuda.is_available():\n    dtype = torch.cuda.FloatTensor\n    torch.set_default_tensor_type('torch.cuda.FloatTensor')\nelse:\n    dtype = torch.FloatTensor\n    torch.set_default_tensor_type('torch.FloatTensor')\n    \nif torch.cuda.is_available():\n    model.cuda()\n        \nif torch.cuda.is_available():\n    perm = perm.cuda()\n"

In [3]:
# choose one grid_index
try:
    grid_index = int(sys.argv[1])
    print('Using Grid index ',grid_index)
except:
    grid_index = 1931
    grid_index = 1259
    print('Using default grid index ',grid_index)

Using default grid index  1259


In [82]:
try:
    grids = Table.read('../../spectrum_grids/galah_dr4_model_trainingset_gridpoints.fits')
    teff_logg_feh_name = str(int(grids['teff_subgrid'][grid_index]))+'_'+"{:.2f}".format(grids['logg_subgrid'][grid_index])+'_'+"{:.2f}".format(grids['fe_h_subgrid'][grid_index])

    training_set = Table.read('../training_input/'+teff_logg_feh_name+'/galah_dr4_trainingset_'+teff_logg_feh_name+'_incl_vsini.fits')
    masks = Table.read('../training_input/'+teff_logg_feh_name+'/'+teff_logg_feh_name+'_masks.fits')

    wavelength_file = '../training_input/galah_dr4_3dbin_wavelength_array.pickle'
    flux_ivar_file = '../training_input/'+teff_logg_feh_name+'/galah_dr4_trainingset_'+teff_logg_feh_name+'_incl_vsini_flux_ivar.pickle'

    flux_ivar_file_opener = open(flux_ivar_file,'rb')
    training_set_flux = pickle.load(flux_ivar_file_opener)
    flux_ivar_file_opener.close()
    wavelength_file_opener = open(wavelength_file,'rb')
    wavelength_array = pickle.load(wavelength_file_opener)
    wavelength_file_opener.close()

    print('Succesfully read in all information needed for grid index '+str(grid_index)+' corresponding to '+teff_logg_feh_name)

except:
    raise ValueError('There are only '+str(len(grids))+' entries within the grid')

Succesfully read in all information needed for grid index 1259 corresponding to 4250_1.50_-0.50


In [83]:
labels = tuple(training_set.keys()[2:-1])

print('Labels to be fitted: ',len(labels))
print(labels)

training_labels = np.array([training_set[label] for label in labels]).T

Labels to be fitted:  36
('teff', 'logg', 'fe_h', 'vmic', 'vsini', 'li_fe', 'c_fe', 'n_fe', 'o_fe', 'na_fe', 'mg_fe', 'al_fe', 'si_fe', 'k_fe', 'ca_fe', 'sc_fe', 'ti_fe', 'v_fe', 'cr_fe', 'mn_fe', 'co_fe', 'ni_fe', 'cu_fe', 'zn_fe', 'rb_fe', 'sr_fe', 'y_fe', 'zr_fe', 'mo_fe', 'ru_fe', 'ba_fe', 'la_fe', 'ce_fe', 'nd_fe', 'sm_fe', 'eu_fe')


In [84]:
model_file = 'galah_dr4_thepayne_model_'+teff_logg_feh_name+'_'+str(len(labels))+'labels'

print('Will create The Payne model to be stored at ')
print('models/'+model_file+'.model')

Will create The Payne model to be stored at 
models/galah_dr4_thepayne_model_4250_1.50_-0.50_36labels.model


In [86]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(np.arange(np.shape(training_set_flux)[0]), test_size=0.2)

In [ ]:
training.neural_net(
    training_labels = training_labels[train,:], 
    training_spectra = training_set_flux[train,:],
    validation_labels = training_labels[test,:], 
    validation_spectra = training_set_flux[test,:],
    num_neurons=300,
    learning_rate=1e-4,
    num_steps=1e4,
    batch_size=np.min([256,np.int(np.shape(training_labels)[1]/2.)]), 
    num_pixel=np.shape(training_set_flux[0])[0],
    training_loss_name = 'models/'+model_file+'_loss.npz',
    payne_model_name = 'models/'+model_file+'.npz'
    )

/Users/svenbuder/anaconda3/lib/python3.7/site-packages/The_Payne-1.1-py3.7.egg/The_Payne/radam.py:48: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1640812107944/work/torch/csrc/utils/python_arg_parser.cpp:1050.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


iter 0: training loss = 8777.473 validation loss = 8659.254
